In [ ]:
## inputs
templateGsheetID = '1ps54IsLGHaExDOfOOr6lYKzPuHmVKKvda1VBZXKE-UI' # this is the ID of the template gsheet
initial_date = date(year=2022, month=6, day=28)

storeListGsheetID = '1j2oKTNI8C5k7Do4ZTyZa-Zsj9agUMe1PXuSM5tkHp-8' # this is the store list gsheet ID - with store numbers, store names and permission emails


In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')


Authenticated
Mounted at /content/drive


In [2]:
!pip install gspread --upgrade # Uninstall gspread-3.0.1 with 3.7.0
import gspread
from gspread.models import Cell, Spreadsheet
from oauth2client.client import GoogleCredentials as GC
import random
import string
from datetime import date, time, datetime, timedelta
gs_ = gspread.authorize(GC.get_application_default())

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [3]:
# # open template gsheet
# template_wb = gs_.open_by_url('https://docs.google.com/spreadsheets/d/1R093LxzAtjIqTBe-vO2NUX9xEXf6IEDtTO6GnTLy9SI/edit#gid=0')
# template_sh = template_wb.get_worksheet(0)

In [4]:
def getGworkbook(ref, client):
  """
  ref: is a gsheet url or key
  """
  try:
    return client.open_by_url(ref)
  except:
    return client.open_by_key(ref)

def genFileName(N):
  # import random
  # import string
  return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(N))

def genWeekEndingsList(initDate, n=52, ascending=True):
  ll = list()
  ll.append(initDate)
  if ascending:
    inc_=7
  else:
    inc_=-7
  for i in range(n):
    dateDiff = timedelta(days=inc_*(i+1))
    date2Append = initDate+dateDiff
    ll.append(date2Append)
  return ll

# def addSheetsToSpreadsheet(gs_spreadsheet,newSheetNames,source_sheet_id=0):
#   for i in range(1,len(newSheetNames)):
#     gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=newSheetNames[i-1])


def addSheetToSpreadsheet(gs_spreadsheet,source_sheet_id, insert_sheet_index, new_sheet_id, new_sheet_name):
  gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=insert_sheet_index, new_sheet_id=new_sheet_id, new_sheet_name=new_sheet_name)




In [6]:
## vars
we_dates = genWeekEndingsList(initial_date,n=52, ascending=False)
we_dates_str = [str(x) for x in we_dates]

In [7]:
# open template gsheet
#template_wb = getGworkbook('https://docs.google.com/spreadsheets/d/1R093LxzAtjIqTBe-vO2NUX9xEXf6IEDtTO6GnTLy9SI/edit#gid=0', gs_)
template_wb = getGworkbook(templateGsheetID, gs_)

In [8]:
# copy template spreadsheet to new location
#!cp /content/drive/My\ Drive/Models/nz_lotto/template_file.gsheet /content/drive/My\ Drive/Models/nz_lotto/copyOf_template_file.gsheet

In [9]:
# # https://stackoverflow.com/questions/61094190/gspread-how-to-duplicate-sheet
# client = gspread.authorize(GC.get_application_default())

# spreadsheet_client = Spreadsheet(client)
# spreadsheet_client.duplicate_sheet("1R093LxzAtjIqTBe-vO2NUX9xEXf6IEDtTO6GnTLy9SI", new_sheet_name="timcard2")
# # worksheet = client.open("timcard2")
# # worksheet.share("my_email@google.com", perm_type='user', role='writer')

In [10]:
client = gspread.authorize(GC.get_application_default())
fName = genFileName(25)
## create new file as copy of template
newGsheet = client.copy(templateGsheetID, title=fName, copy_permissions=True)
print("File "+fName+" created.")


File 9HJY69DZHL25OLM15CRCHIO7H created.


In [11]:
# add a new sheet 1 for each week
# for i in range(1,len(we_dates_str)):
#   newGsheet.duplicate_sheet(source_sheet_id=0, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=we_dates_str[i-1])

# addSheetsToSpreadsheet(gs_spreadsheet=newGsheet,newSheetNames=we_dates_str,source_sheet_id=0)

for i in range(1,len(we_dates_str)):
  i_we_dates_str = we_dates_str[i-1]
  addSheetToSpreadsheet(gs_spreadsheet=newGsheet,source_sheet_id=0, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=i_we_dates_str)
  worksheet = newGsheet.worksheet(i_we_dates_str)
  #newGsheet.duplicate_sheet(source_sheet_id=0, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=we_dates_str[i-1])
  # batch update to avoid api usage limits.
  # worksheet.batch_update([{
  #     'range': 'C10',
  #     'values': [[i_we_dates_str]],
  # }, {
  #     'range': 'D10',
  #     'values': [['1234']],
  # }])
  worksheet.update_acell('F2',i_we_dates_str)

  # worksheet.format("F2", {
  #     "numberFormat": {
  #         "type": "DATE"
  #         }
  #     })



In [12]:
# # for each sheet in spreadsheet write to a cell.
# dict_cell_values{'b1':}
# worksheet = newGsheet.worksheet('2022-06-28')
# worksheet.update_acell('A1','=2+3')


SyntaxError: ignored

In [ ]:
# batch update? to avoid api usage limits.

# worksheet.batch_update([{
#     'range': 'A1:B1',
#     'values': [['42', '43']],
# }, {
#     'range': 'my_range',
#     'values': [['44', '45']],
# }])


In [ ]:
# # https://docs.gspread.org/en/latest/api.html#gspread.models.Spreadsheet.share
# worksheet.spreadsheet.share('jcorderoy@woolworths.com.au', perm_type='user', role='owner', notify=True, email_message=None, with_link=False)

In [ ]:
# worksheet.spreadsheet.list_permissions()
# [x['emailAddress'] for x in worksheet.spreadsheet.list_permissions()]
# worksheet.spreadsheet.remove_permissions('bwahl@woolworths.com.au')

In [13]:
# get store list gsheet
import numpy as np
storeList_worksheet = getGworkbook(storeListGsheetID, gs_).sheet1
storeList_array = np.array(storeList_worksheet.get_all_values())[1:]

store_nums = storeList_array.T[0]
store_gsheet_names = storeList_array.T[1]
store_name_in_gsheet = storeList_array.T[2]

# clean access names:
access_name_separator = ';'
cleaned_perm_emails = [x[3].split(access_name_separator) for x in storeList_array]

ll = []
for i in range(len(cleaned_perm_emails)):
  ll.append( [s.strip() for s in cleaned_perm_emails[i]] )
cleaned_perm_emails = ll

In [ ]:
cleaned_perm_emails


In [22]:
## protect a cell
worksheet.add_protected_range('A1')

{'replies': [{'addProtectedRange': {'protectedRange': {'editors': {'users': ['bwahl@woolworths.com.au',
       'jcorderoy@woolworths.com.au']},
     'protectedRangeId': 1548911740,
     'range': {'endColumnIndex': 1,
      'endRowIndex': 1,
      'sheetId': 52,
      'startColumnIndex': 0,
      'startRowIndex': 0},
     'requestingUserCanEdit': True}}}],
 'spreadsheetId': '1c89aWt14mik-YmiNZAoJkO7sHdHUq5oLUHvcbO8q-bQ'}